####This notebook walks through the approach 1 : Model with all features
- to run this notebook please have the dataset "category_data" uploaded
- While uploading the above dataset make sure to set the schema as the following 
- Schema : _Date:timestamp, d:double, wm_yr_wk:double, weekday:string, month:double, year:double, Hobbies:double, Foods:double, HouseHold:double, event_name_1:string, event_type_1:string, event_name_2:string, event_type_2:string_

In [2]:
import random
random.seed(50)

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from datetime import datetime
from pyspark.sql.functions import udf, split, when, isnan, count, col, monotonically_increasing_id
from pyspark.sql.types import DateType, IntegerType, DoubleType, FloatType, StringType
from pyspark.sql import *
from pyspark.sql.window import Window
from pyspark.sql import functions as f
from sklearn.metrics import mean_squared_error
from pyspark.ml.feature import *
from pyspark.ml.feature import CountVectorizer, VectorIndexer, VectorAssembler,StringIndexer, OneHotEncoder, VectorSlicer
from pyspark.ml import Pipeline, feature
from pyspark.ml.regression import DecisionTreeRegressor, LinearRegression, RandomForestRegressor, GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.linalg import Vectors
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.base import Estimator
from pyspark.ml.param import Params, Param, TypeConverters
from pyspark.ml.param.shared import HasOutputCol

#####User defined functions

In [4]:
#counting null values 
def check_null(dataset):
  null_count = dataset.select([count(when(col(c).isNull(), c)).alias(c) for c in dataset.columns])
  return (null_count.show())

# Convert data in column to an array so as to use with Count Vectorizer
def convert_col_to_array(df,new_col,orig_col, split_char):
  return(df.withColumn(new_col,split(col(orig_col),split_char)))

# Count Vectorizer
def initialize_Count_Vectorizer(input_Col, output_Col):
  return(CountVectorizer(inputCol = input_Col, outputCol = output_Col))

#SMape error metric
def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

#####Importing Data

In [6]:
#importing the data : 
df = spark.table("category_data")
display(df)

Date,d,wm_yr_wk,weekday,month,year,Hobbies,Foods,HouseHold,event_name_1,event_type_1,event_name_2,event_type_2
2011-01-29T00:00:00.000+0000,1.0,11101.0,Saturday,1.0,2011.0,3764.0,23178.0,5689.0,null,null,null,null
2011-01-30T00:00:00.000+0000,2.0,11101.0,Sunday,1.0,2011.0,3357.0,22758.0,5634.0,null,null,null,null
2011-01-31T00:00:00.000+0000,3.0,11101.0,Monday,1.0,2011.0,2682.0,17174.0,3927.0,null,null,null,null
2011-02-01T00:00:00.000+0000,4.0,11101.0,Tuesday,2.0,2011.0,2669.0,18878.0,3865.0,null,null,null,null
2011-02-02T00:00:00.000+0000,5.0,11101.0,Wednesday,2.0,2011.0,1814.0,14603.0,2729.0,null,null,null,null
2011-02-03T00:00:00.000+0000,6.0,11101.0,Thursday,2.0,2011.0,3220.0,22093.0,3898.0,null,null,null,null
2011-02-04T00:00:00.000+0000,7.0,11101.0,Friday,2.0,2011.0,2944.0,20490.0,4576.0,null,null,null,null
2011-02-05T00:00:00.000+0000,8.0,11102.0,Saturday,2.0,2011.0,3986.0,27751.0,6195.0,null,null,null,null
2011-02-06T00:00:00.000+0000,9.0,11102.0,Sunday,2.0,2011.0,2899.0,24862.0,4975.0,SuperBowl,Sporting,null,null
2011-02-07T00:00:00.000+0000,10.0,11102.0,Monday,2.0,2011.0,2615.0,18901.0,4056.0,null,null,null,null


##### Checking & replacing nulls in the given dataset
- The null values in columns event_name_1 and 2 and event_type_1 and 2 represents that there were no events hence they are replaced with string " No event"

In [8]:
print("Null values in the dataframe")
check_null(df)
print("                                                      ")
print("After replaing Null values in the dataframe")
#replacing null with string value 'No_event' in respective columns
df = df.na.fill({'event_name_1': 'No event','event_name_2': 'No event','event_type_1': 'No event','event_type_2': 'No event' })
check_null(df)

Null values in the dataframe
+----+---+--------+-------+-----+----+-------+-----+---------+------------+------------+------------+------------+
Date| d|wm_yr_wk|weekday|month|year|Hobbies|Foods|HouseHold|event_name_1|event_type_1|event_name_2|event_type_2|
+----+---+--------+-------+-----+----+-------+-----+---------+------------+------------+------------+------------+
 0| 0| 0| 0| 0| 0| 0| 0| 0| 1783| 1783| 1937| 1937|
+----+---+--------+-------+-----+----+-------+-----+---------+------------+------------+------------+------------+

 
After replaing Null values in the dataframe
+----+---+--------+-------+-----+----+-------+-----+---------+------------+------------+------------+------------+
Date| d|wm_yr_wk|weekday|month|year|Hobbies|Foods|HouseHold|event_name_1|event_type_1|event_name_2|event_type_2|
+----+---+--------+-------+-----+----+-------+-----+---------+------------+------------+------------+------------+
 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
+----+---+--------+-------+-----+----+-------+-----+---------+------------+------------+------------+------------+

#####Feature Generation
- "no_of_events" - This column gives the total number of events for each day by summing the columns event_name_1 and 2
- More features are generated below

In [10]:
def count_events(c1, c2):
  if c1 != "No event" and c2 != "No event":
    return 2
  elif c1 == "No event" and c2 != "No event":
    return 1
  elif c1 != "No event" and c2 == "No event":
    return 1
  else:
    return 0

combineUDF = udf(count_events)
expr = [c for c in df] + [combineUDF(col("event_name_1"), col("event_name_2")).alias("no_of_events").cast(DoubleType())]
df = df.select(*expr)

In [11]:
display(df)

Date,d,wm_yr_wk,weekday,month,year,Hobbies,Foods,HouseHold,event_name_1,event_type_1,event_name_2,event_type_2,no_of_events
2011-01-29T00:00:00.000+0000,1.0,11101.0,Saturday,1.0,2011.0,3764.0,23178.0,5689.0,No event,No event,No event,No event,0.0
2011-01-30T00:00:00.000+0000,2.0,11101.0,Sunday,1.0,2011.0,3357.0,22758.0,5634.0,No event,No event,No event,No event,0.0
2011-01-31T00:00:00.000+0000,3.0,11101.0,Monday,1.0,2011.0,2682.0,17174.0,3927.0,No event,No event,No event,No event,0.0
2011-02-01T00:00:00.000+0000,4.0,11101.0,Tuesday,2.0,2011.0,2669.0,18878.0,3865.0,No event,No event,No event,No event,0.0
2011-02-02T00:00:00.000+0000,5.0,11101.0,Wednesday,2.0,2011.0,1814.0,14603.0,2729.0,No event,No event,No event,No event,0.0
2011-02-03T00:00:00.000+0000,6.0,11101.0,Thursday,2.0,2011.0,3220.0,22093.0,3898.0,No event,No event,No event,No event,0.0
2011-02-04T00:00:00.000+0000,7.0,11101.0,Friday,2.0,2011.0,2944.0,20490.0,4576.0,No event,No event,No event,No event,0.0
2011-02-05T00:00:00.000+0000,8.0,11102.0,Saturday,2.0,2011.0,3986.0,27751.0,6195.0,No event,No event,No event,No event,0.0
2011-02-06T00:00:00.000+0000,9.0,11102.0,Sunday,2.0,2011.0,2899.0,24862.0,4975.0,SuperBowl,Sporting,No event,No event,1.0
2011-02-07T00:00:00.000+0000,10.0,11102.0,Monday,2.0,2011.0,2615.0,18901.0,4056.0,No event,No event,No event,No event,0.0


In [12]:
# dropping columns we won't be considering for model development
cat_df = df.drop("Date","event_name_1","event_name_2")

#####One hot encoding of categorical columns
- The columns weekday, month, year, event_type_1 and event_type_2 are one hot encoded

In [14]:
#One Hot Encoding of categorical columns

OHE_list= ['weekday','month','year','event_type_1','event_type_2']

# Generate OHE for elements in the list above
for element in OHE_list:
  print(element)
  
#Convert String to Array so that OHE can run (requires an array as an input)
  cat_df = convert_col_to_array(cat_df, element + '_array', element,' ')
  
#Initialize Count vectorizer
  elementVectorizer = initialize_Count_Vectorizer(element + '_array',element + '_OHE')
  
#Fit a vectorizer model
  elementVectorizer_model = elementVectorizer.fit(cat_df)
  
#Transform Data
  cat_df = elementVectorizer_model.transform(cat_df)
  
#Specify Extraneous Cols to drop 
  columns_to_drop = [element, element + '_array']
  
#Drop Extraneous Cols listed above
  cat_df = cat_df.drop(*columns_to_drop)

weekday
month
year
event_type_1
event_type_2

In [15]:
#dataset after OHE
display(cat_df)

d,wm_yr_wk,Hobbies,Foods,HouseHold,no_of_events,weekday_OHE,month_OHE,year_OHE,event_type_1_OHE,event_type_2_OHE
1.0,11101.0,3764.0,23178.0,5689.0,0.0,"List(0, 7, List(1), List(1.0))","List(0, 12, List(4), List(1.0))","List(0, 6, List(4), List(1.0))","List(0, 6, List(0, 1), List(1.0, 1.0))","List(0, 4, List(0, 1), List(1.0, 1.0))"
2.0,11101.0,3357.0,22758.0,5634.0,0.0,"List(0, 7, List(0), List(1.0))","List(0, 12, List(4), List(1.0))","List(0, 6, List(4), List(1.0))","List(0, 6, List(0, 1), List(1.0, 1.0))","List(0, 4, List(0, 1), List(1.0, 1.0))"
3.0,11101.0,2682.0,17174.0,3927.0,0.0,"List(0, 7, List(2), List(1.0))","List(0, 12, List(4), List(1.0))","List(0, 6, List(4), List(1.0))","List(0, 6, List(0, 1), List(1.0, 1.0))","List(0, 4, List(0, 1), List(1.0, 1.0))"
4.0,11101.0,2669.0,18878.0,3865.0,0.0,"List(0, 7, List(6), List(1.0))","List(0, 12, List(3), List(1.0))","List(0, 6, List(4), List(1.0))","List(0, 6, List(0, 1), List(1.0, 1.0))","List(0, 4, List(0, 1), List(1.0, 1.0))"
5.0,11101.0,1814.0,14603.0,2729.0,0.0,"List(0, 7, List(3), List(1.0))","List(0, 12, List(3), List(1.0))","List(0, 6, List(4), List(1.0))","List(0, 6, List(0, 1), List(1.0, 1.0))","List(0, 4, List(0, 1), List(1.0, 1.0))"
6.0,11101.0,3220.0,22093.0,3898.0,0.0,"List(0, 7, List(5), List(1.0))","List(0, 12, List(3), List(1.0))","List(0, 6, List(4), List(1.0))","List(0, 6, List(0, 1), List(1.0, 1.0))","List(0, 4, List(0, 1), List(1.0, 1.0))"
7.0,11101.0,2944.0,20490.0,4576.0,0.0,"List(0, 7, List(4), List(1.0))","List(0, 12, List(3), List(1.0))","List(0, 6, List(4), List(1.0))","List(0, 6, List(0, 1), List(1.0, 1.0))","List(0, 4, List(0, 1), List(1.0, 1.0))"
8.0,11102.0,3986.0,27751.0,6195.0,0.0,"List(0, 7, List(1), List(1.0))","List(0, 12, List(3), List(1.0))","List(0, 6, List(4), List(1.0))","List(0, 6, List(0, 1), List(1.0, 1.0))","List(0, 4, List(0, 1), List(1.0, 1.0))"
9.0,11102.0,2899.0,24862.0,4975.0,1.0,"List(0, 7, List(0), List(1.0))","List(0, 12, List(3), List(1.0))","List(0, 6, List(4), List(1.0))","List(0, 6, List(5), List(1.0))","List(0, 4, List(0, 1), List(1.0, 1.0))"
10.0,11102.0,2615.0,18901.0,4056.0,0.0,"List(0, 7, List(2), List(1.0))","List(0, 12, List(3), List(1.0))","List(0, 6, List(4), List(1.0))","List(0, 6, List(0, 1), List(1.0, 1.0))","List(0, 4, List(0, 1), List(1.0, 1.0))"


#####Defining Time Horizons
The datasets are prepared and subsetted for the over three time horizon for each of the three catergories
1. Day
2. Week
3. Month

In [17]:
#function for creating lags in the dataset

def time_horizon(input_df, lag):
  window = Window.orderBy("d")
  output_df_name = input_df.withColumn("lag_Hobbies",f.lag(col("Hobbies"),lag).over(window)).withColumn("lag_Foods",f.lag(col("Foods"),lag).over(window)).withColumn("lag_HH",f.lag(col("HouseHold"),lag).over(window)).drop("Hobbies","Foods","HouseHold")
  return(output_df_name)

day_df = time_horizon(cat_df,-1)
week_df = time_horizon(cat_df,-7)
month_df = time_horizon(cat_df,-28)

In [18]:
#Formulating target specific datasets
#Subsetting dataframes for day, week, month over all categories
def sub_data(dataset,category):
  list1 = ['lag_Foods', 'lag_HH']
  list2 = ['lag_Hobbies', 'lag_HH']
  list3 = ['lag_Hobbies', 'lag_Foods']
  Hobbies_x = dataset.drop(*list1)
  Foods_x = dataset.drop(*list2)
  HH_x = dataset.drop(*list3)
  if category == "Hobbies":
     return (Hobbies_x)
  elif category == "Foods":
     return (Foods_x)
  elif category == "HouseHold":
     return (HH_x)
  else:
    return(print("Enter either Hobbies, Foods or HouseHold"))

Hobbies_day = sub_data(day_df,"Hobbies")
Foods_day = sub_data(day_df,"Foods")
HH_day = sub_data(day_df,"HouseHold")

Hobbies_week = sub_data(week_df,"Hobbies")
Foods_week = sub_data(week_df,"Foods")
HH_week = sub_data(week_df,"HouseHold")

Hobbies_month = sub_data(month_df,"Hobbies")
Foods_month = sub_data(month_df,"Foods")
HH_month = sub_data(month_df,"HouseHold")

#####Feature Generation
The following 10 features are generated representing the sales quantity in past for different time range,
>>1. __Pre_d1__ : the sales quantity one day before the target
2.  __Pre_d2__ : the sales quantity two days before the target
3.  __Pre_d3__ : the sales quantity three days before the target
4.  __Pre_d4__ : the sales quantity four days before the target
5.  __Pre_d5__ : the sales quantity five days before the target
6.  __Pre_d6__ : the sales quantity six days before the target
7.  __Pre_d7__ : the sales quantity seven days before the target
8.  __Pre_d10__ : the sales quantity ten days before the target
9.  __Pre_d14__ : the sales quantity fourteen days before the target
10.  __Pre_d28__ : the sales quantity twenty eight days before the target

All of these generated features along with rest of features already available will be passed through a feature selector and the model will be implemented with the auto  selected features

In [20]:
def generate_feat(df1,df2,column):
    window = Window.orderBy("d")
    x = df1.withColumn("Pre_d1",f.lag(col(column),1).over(window)).withColumn("Pre_d2",f.lag(col(column),2).over(window)).withColumn("Pre_d3",f.lag(col(column),3).over(window)).withColumn("Pre_d4",f.lag(col(column),4).over(window)).withColumn("Pre_d5",f.lag(col(column),5).over(window)).withColumn("Pre_d6",f.lag(col(column),6).over(window)).withColumn("Pre_d7",f.lag(col(column),7).over(window)).withColumn("Pre_d10",f.lag(col(column),10).over(window)).withColumn("Pre_d14",f.lag(col(column),14).over(window)).withColumn("Pre_d28",f.lag(col(column),28).over(window))
    x_sub = x.select("d","Pre_d1","Pre_d2","Pre_d3","Pre_d4","Pre_d5","Pre_d6","Pre_d7","Pre_d10","Pre_d14","Pre_d28")
    df2 = df2.join(x_sub, x_sub.d == df2.d).drop("d")
    return(df2)
 
Hob_day = generate_feat(cat_df,Hobbies_day,"Hobbies").dropna()
F_day   = generate_feat(cat_df,Foods_day,"Foods").dropna()
HH_day  = generate_feat(cat_df,HH_day,"HouseHold").dropna()

Hob_week = generate_feat(cat_df,Hobbies_week,"Hobbies").dropna()
F_week   = generate_feat(cat_df,Foods_week,"Foods").dropna()
HH_week  = generate_feat(cat_df,HH_week,"HouseHold").dropna()

Hob_month = generate_feat(cat_df,Hobbies_month,"Hobbies").dropna()
F_month = generate_feat(cat_df,Foods_month,"Foods").dropna()
HH_month = generate_feat(cat_df,HH_month,"HouseHold").dropna()

##### Model implemetation
###### Predicting over each category for the time horizons 1 day, 7 days and 28 days using the following algorithm
1. Gradient Boosting
2. Random Forest

NOTE : 
- The following set of code for each algorithm involves a increasing rolling window train test split with a split of 3
- Further Hyper parameter tuning is performed using Grid search and the parameters which give the least error value is chosen

###### HOBBIES : 1 DAY

In [23]:
# Specify Input Columns that will become features in our final models
featuresCols = Hob_day.columns
featuresCols.remove('lag_Hobbies')

#Build Features using the feature assembler
feature_assembler = VectorAssembler(inputCols = featuresCols, outputCol = "features")
output = feature_assembler.transform(Hob_day) # Transform dataframe into features
final_Hobbies = output.select("features", "lag_Hobbies") # Generate final output two cols: one with features and the other with the target i.e Hobbies

#Rename Hobbies Column to Hobbies_quantity to use with ML models in PySpark
final_Hobbies = final_Hobbies.select(col("features"),col("lag_Hobbies").alias("Hobbies_quantity"))

#Build Feature Indexer:
feature_Indexer = VectorIndexer(inputCol = 'features', outputCol = 'indexedFeatures',maxCategories = 12).fit(final_Hobbies)

final_Hobbies = final_Hobbies.select("*").withColumn("id", monotonically_increasing_id())

In [24]:
#GBT for Hobbies 1 day horizon
import numpy as np
import pandas as pd
Hobbies = final_Hobbies.drop('id')
X = np.array(Hobbies.select('features','Hobbies_quantity').collect())

from sklearn.model_selection import TimeSeriesSplit
from sklearn import datasets, ensemble
splits = TimeSeriesSplit(n_splits=3)
maxDepth = (2,5)
maxIter = (10,20)
GBT_feature_list = []
for train_index, test_index in splits.split(X):                
  train = X[train_index]
  test = X[test_index]
  train_df_pyspark = final_Hobbies.where(col("id").between(0, len(train))).drop('id')
  test_df_pyspark = final_Hobbies.where(col("id").between(len(train)+1, len(train)+len(test))).drop('id')
  for i in maxDepth:
    for j in maxIter:
      GBT = GBTRegressor(labelCol="Hobbies_quantity",maxDepth = i, maxIter = j)
      pipeline = Pipeline(stages = [feature_Indexer, GBT])
      model = pipeline.fit(train_df_pyspark)
      predictions = model.transform(test_df_pyspark)
      e = RegressionEvaluator(metricName="r2", labelCol=GBT.getLabelCol(), predictionCol=GBT.getPredictionCol())
      r2= e.evaluate(predictions)
      A = np.array(predictions.select('Hobbies_quantity').collect())
      F = np.array(predictions.select('prediction').collect())
      #Print RMSE for the current regressor along with the regression model  
      s = smape(A,F)
      f = [i,j,r2,s]
      GBT_feature_list.append(f)
GBT_feature_list 

Out[15]: [[2, 10, 0.4439999298513443, 13.769348199352144],
 [2, 20, 0.42684532404118347, 13.880732240044178],
 [5, 10, 0.16240541985809254, 17.000009479398443],
 [5, 20, 0.14154839751839732, 17.101336657775928],
 [2, 10, 0.3819215681475798, 9.347559426967035],
 [2, 20, 0.4207412293338122, 9.004248160776116],
 [5, 10, 0.3774449058680315, 9.717721699115023],
 [5, 20, 0.36181317555864956, 9.971840299840954],
 [2, 10, -0.017188470241485954, 11.887646594279243],
 [2, 20, 0.08451940304351391, 11.460194822065368],
 [5, 10, -0.9155863922026282, 19.234481955866972],
 [5, 20, -1.1188115469536313, 20.485993963300213]]

In [25]:
#RF for Hobbies 1 day horizon
Hobbies = final_Hobbies.drop('id')
X = np.array(Hobbies.select('features','Hobbies_quantity').collect())

from sklearn.model_selection import TimeSeriesSplit
from sklearn import datasets, ensemble
splits = TimeSeriesSplit(n_splits=3)
numTrees = [5,15,30,50]
maxDepth = [5,10,15]
RF_feature_list = []
for train_index, test_index in splits.split(X):
  train = X[train_index]
  test = X[test_index]
  train_df_pyspark = final_Hobbies.where(col("id").between(0, len(train))).drop('id')
  test_df_pyspark = final_Hobbies.where(col("id").between(len(train)+1, len(train)+len(test))).drop('id')
  for i in numTrees:
    for j in maxDepth:
      RF = RandomForestRegressor(labelCol="Hobbies_quantity",numTrees = i, maxDepth = j)
      pipeline = Pipeline(stages = [feature_Indexer, RF])
      model = pipeline.fit(train_df_pyspark)
      predictions = model.transform(test_df_pyspark)
      e = RegressionEvaluator(metricName="r2", labelCol=RF.getLabelCol(), predictionCol=RF.getPredictionCol())
      r2= e.evaluate(predictions)
      A = np.array(predictions.select('Hobbies_quantity').collect())
      F = np.array(predictions.select('prediction').collect())
      #Print RMSE for the current regressor along with the regression model  
      s = smape(A,F)
      f = [i,j,r2,s]
      RF_feature_list.append(f)
RF_feature_list

Out[16]: [[5, 5, 0.3769645299258928, 14.52893177562492],
 [5, 10, 0.3719861150476452, 14.560858945647956],
 [5, 15, 0.37331331016142044, 14.523168364065803],
 [15, 5, 0.2829603362938299, 15.723621612605255],
 [15, 10, 0.28463465927625076, 15.573394059673554],
 [15, 15, 0.28184553939480483, 15.664548066241958],
 [30, 5, 0.2909480664523073, 15.710769643441125],
 [30, 10, 0.32326656671392706, 15.134757766644109],
 [30, 15, 0.3230655869110576, 15.177041543929361],
 [50, 5, 0.37453738057405594, 14.669283775719821],
 [50, 10, 0.38773378637672096, 14.261079222078308],
 [50, 15, 0.3901380230661665, 14.179461536955138],
 [5, 5, 0.482702435986487, 8.293886195459354],
 [5, 10, 0.4131830973846765, 9.52391077249618],
 [5, 15, 0.40851695456414827, 9.722293144666445],
 [15, 5, 0.47853894586992396, 8.23968139953699],
 [15, 10, 0.46673976590801836, 8.764818531995857],
 [15, 15, 0.46673411968929324, 8.76236816726899],
 [30, 5, 0.5107248139188689, 8.162205438939685],
 [30, 10, 0.5258538543671012, 7.9995986627303095],
 [30, 15, 0.5261577145304321, 8.033203800798955],
 [50, 5, 0.5153293219758948, 8.094270785514302],
 [50, 10, 0.544295949510722, 7.773507980824496],
 [50, 15, 0.5397332188477004, 7.845581024734891],
 [5, 5, 0.21813070678230773, 10.040512847754389],
 [5, 10, -0.35150422056861363, 14.956311155530754],
 [5, 15, -0.3303384677131016, 14.716105647861045],
 [15, 5, -0.07822683831867305, 13.048248726282173],
 [15, 10, -0.11184272085305724, 13.545909236626285],
 [15, 15, -0.12847455371598793, 13.712025381695454],
 [30, 5, -0.25154309558337196, 14.811488612557582],
 [30, 10, -0.38310052828492, 15.973412767263557],
 [30, 15, -0.37294807080719594, 15.863439372533158],
 [50, 5, -0.2154688426357787, 14.497687850542544],
 [50, 10, -0.5790513073236594, 17.54970341325132],
 [50, 15, -0.587774010586205, 17.565692230311313]]

###### HOBBIES : 7 DAY

In [27]:
# Specify Input Columns that will become features in our final models
featuresCols = Hob_week.columns
featuresCols.remove('lag_Hobbies')

#Build Features using the feature assembler
feature_assembler = VectorAssembler(inputCols = featuresCols, outputCol = "features")
output = feature_assembler.transform(Hob_week) # Transform dataframe into features
final_Hobbies = output.select("features", "lag_Hobbies") # Generate final output two cols: one with features and the other with the target i.e Hobbies

#Rename Hobbies Column to Hobbies_quantity to use with ML models in PySpark
final_Hobbies = final_Hobbies.select(col("features"),col("lag_Hobbies").alias("Hobbies_quantity"))

#Build Feature Indexer:
feature_Indexer = VectorIndexer(inputCol = 'features', outputCol = 'indexedFeatures',maxCategories = 12).fit(final_Hobbies)

final_Hobbies = final_Hobbies.select("*").withColumn("id", monotonically_increasing_id())

In [28]:
#GBT for Hobbies 7 days horizon
Hobbies = final_Hobbies.drop('id')
X = np.array(Hobbies.select('features','Hobbies_quantity').collect())

from sklearn.model_selection import TimeSeriesSplit
from sklearn import datasets, ensemble
splits = TimeSeriesSplit(n_splits=3)
maxDepth = (2,5)
maxIter = (10,20)
GBT_feature_list = []
for train_index, test_index in splits.split(X):                
  train = X[train_index]
  test = X[test_index]
  train_df_pyspark = final_Hobbies.where(col("id").between(0, len(train))).drop('id')
  test_df_pyspark = final_Hobbies.where(col("id").between(len(train)+1, len(train)+len(test))).drop('id')
  for i in maxDepth:
    for j in maxIter:
      GBT = GBTRegressor(labelCol="Hobbies_quantity",maxDepth = i, maxIter = j)
      pipeline = Pipeline(stages = [feature_Indexer, GBT])
      model = pipeline.fit(train_df_pyspark)
      predictions = model.transform(test_df_pyspark)
      e = RegressionEvaluator(metricName="r2", labelCol=GBT.getLabelCol(), predictionCol=GBT.getPredictionCol())
      r2= e.evaluate(predictions)
      A = np.array(predictions.select('Hobbies_quantity').collect())
      F = np.array(predictions.select('prediction').collect())
      #Print RMSE for the current regressor along with the regression model  
      s = smape(A,F)
      f = [i,j,r2,s]
      GBT_feature_list.append(f)
GBT_feature_list 

Out[21]: [[2, 10, 0.0762419794942617, 19.444649544542045],
 [2, 20, 0.12189335976289994, 19.094330709640655],
 [5, 10, -3.4665323910516, 45.70003782315181],
 [5, 20, -3.4973866730707757, 45.82781408239148],
 [2, 10, 0.39736256736622466, 9.668475739007391],
 [2, 20, 0.3904090104662693, 9.62581523427639],
 [5, 10, 0.26870339626465234, 11.290272536460034],
 [5, 20, 0.24370259232062763, 11.404096346188803],
 [2, 10, -0.006897964237520204, 11.957484626752489],
 [2, 20, 0.05193381958183385, 11.519701758722334],
 [5, 10, 0.11049843094996104, 10.996326240996547],
 [5, 20, 0.06096485750280278, 11.446208011970823]]

In [29]:
#RF for Hobbies 7 days horizon
Hobbies = final_Hobbies.drop('id')
X = np.array(Hobbies.select('features','Hobbies_quantity').collect())

from sklearn.model_selection import TimeSeriesSplit
from sklearn import datasets, ensemble
splits = TimeSeriesSplit(n_splits=3)
numTrees = [5,15,30,50]
maxDepth = [5,10,15]
RF_feature_list = []
for train_index, test_index in splits.split(X):
  train = X[train_index]
  test = X[test_index]
  train_df_pyspark = final_Hobbies.where(col("id").between(0, len(train))).drop('id')
  test_df_pyspark = final_Hobbies.where(col("id").between(len(train)+1, len(train)+len(test))).drop('id')
  for i in numTrees:
    for j in maxDepth:
      RF = RandomForestRegressor(labelCol="Hobbies_quantity",numTrees = i, maxDepth = j)
      pipeline = Pipeline(stages = [feature_Indexer, RF])
      model = pipeline.fit(train_df_pyspark)
      predictions = model.transform(test_df_pyspark)
      e = RegressionEvaluator(metricName="r2", labelCol=RF.getLabelCol(), predictionCol=RF.getPredictionCol())
      r2= e.evaluate(predictions)
      A = np.array(predictions.select('Hobbies_quantity').collect())
      F = np.array(predictions.select('prediction').collect())
      #Print RMSE for the current regressor along with the regression model  
      s = smape(A,F)
      f = [i,j,r2,s]
      RF_feature_list.append(f)
RF_feature_list

Out[18]: [[5, 5, 0.09802685109578668, 18.48659959297158],
 [5, 10, 0.049320931771472454, 19.400126246317626],
 [5, 15, 0.05124749194120315, 19.42386920507776],
 [15, 5, 0.08302410336440857, 18.704436114777355],
 [15, 10, 0.035718395332933306, 19.214765935375176],
 [15, 15, 0.02737277030365992, 19.301625876251368],
 [30, 5, 0.24928448646460533, 16.81279830342623],
 [30, 10, 0.23363177184073247, 17.045773558099306],
 [30, 15, 0.229742133722722, 17.115851517398095],
 [50, 5, 0.1576308406611806, 17.790940622428383],
 [50, 10, 0.1169544871549556, 18.393595540112912],
 [50, 15, 0.11347233251297639, 18.42699248410858],
 [5, 5, 0.4083946953128842, 9.479458796445305],
 [5, 10, 0.312221880280652, 10.457554875366307],
 [5, 15, 0.3135739921369183, 10.564963590346398],
 [15, 5, 0.4863181361632599, 8.576955723248387],
 [15, 10, 0.4425934499911286, 8.977389191964319],
 [15, 15, 0.44511259710296036, 9.008286991492097],
 [30, 5, 0.49565513076292433, 8.428968740489566],
 [30, 10, 0.4949162123779386, 8.428823476756952],
 [30, 15, 0.48764016091043005, 8.485849611404745],
 [50, 5, 0.5202396610195439, 8.15666048290885],
 [50, 10, 0.5187209763460453, 8.29256805906468],
 [50, 15, 0.5179799329076058, 8.31532687386762],
 [5, 5, 0.1763115710751364, 10.39352701003506],
 [5, 10, 0.03282869575739922, 11.7265287201674],
 [5, 15, -0.043966891181428824, 12.18959317518592],
 [15, 5, 0.15121801136288615, 10.811841499978566],
 [15, 10, 0.21754971579702076, 10.370332106886973],
 [15, 15, 0.1941295631590254, 10.647992029517164],
 [30, 5, 0.2772977632986994, 9.633137634301162],
 [30, 10, 0.2782624223912439, 9.837328238322486],
 [30, 15, 0.2820419951077884, 9.816206928984714],
 [50, 5, 0.20017966738383142, 10.457266450128763],
 [50, 10, 0.18718946605286269, 10.750556999233858],
 [50, 15, 0.1881366004365107, 10.79341662338506]]

###### HOBBIES : 28 DAY

In [31]:
# Specify Input Columns that will become features in our final models
featuresCols = Hob_month.columns
featuresCols.remove('lag_Hobbies')

#Build Features using the feature assembler
feature_assembler = VectorAssembler(inputCols = featuresCols, outputCol = "features")
output = feature_assembler.transform(Hob_month) # Transform dataframe into features
final_Hobbies = output.select("features", "lag_Hobbies") # Generate final output two cols: one with features and the other with the target i.e Hobbies

#Rename Hobbies Column to Hobbies_quantity to use with ML models in PySpark
final_Hobbies = final_Hobbies.select(col("features"),col("lag_Hobbies").alias("Hobbies_quantity"))

#Build Feature Indexer:
feature_Indexer = VectorIndexer(inputCol = 'features', outputCol = 'indexedFeatures',maxCategories = 12).fit(final_Hobbies)

final_Hobbies = final_Hobbies.select("*").withColumn("id", monotonically_increasing_id())

In [32]:
#GBT for Hobbies 28 days horizon
Hobbies = final_Hobbies.drop('id')
X = np.array(Hobbies.select('features','Hobbies_quantity').collect())

from sklearn.model_selection import TimeSeriesSplit
from sklearn import datasets, ensemble
splits = TimeSeriesSplit(n_splits=3)
maxDepth = (2,5)
maxIter = (10,20)
GBT_feature_list = []
for train_index, test_index in splits.split(X):                
  train = X[train_index]
  test = X[test_index]
  train_df_pyspark = final_Hobbies.where(col("id").between(0, len(train))).drop('id')
  test_df_pyspark = final_Hobbies.where(col("id").between(len(train)+1, len(train)+len(test))).drop('id')
  for i in maxDepth:
    for j in maxIter:
      GBT = GBTRegressor(labelCol="Hobbies_quantity",maxDepth = i, maxIter = j)
      pipeline = Pipeline(stages = [feature_Indexer, GBT])
      model = pipeline.fit(train_df_pyspark)
      predictions = model.transform(test_df_pyspark)
      e = RegressionEvaluator(metricName="r2", labelCol=GBT.getLabelCol(), predictionCol=GBT.getPredictionCol())
      r2= e.evaluate(predictions)
      A = np.array(predictions.select('Hobbies_quantity').collect())
      F = np.array(predictions.select('prediction').collect())
      #Print RMSE for the current regressor along with the regression model  
      s = smape(A,F)
      f = [i,j,r2,s]
      GBT_feature_list.append(f)
GBT_feature_list 

Out[24]: [[2, 10, 0.26711913547385635, 16.591028950439185],
 [2, 20, 0.29879564784929336, 16.392651917323356],
 [5, 10, 0.252769123762955, 16.911469052978678],
 [5, 20, 0.25462155378001505, 16.83305284040748],
 [2, 10, 0.28806053115009, 10.532615511612557],
 [2, 20, 0.33170110270796227, 10.002979628539007],
 [5, 10, 0.19875952625317794, 11.456160115324298],
 [5, 20, 0.19750259379661217, 11.523105983925046],
 [2, 10, 0.19555609850026123, 10.605877894524985],
 [2, 20, 0.2802006605396882, 9.809598817344812],
 [5, 10, 0.2140006769134415, 10.277399163824468],
 [5, 20, 0.16244335453828063, 10.745023903025615]]

In [33]:
#RF for Hobbies 28 days horizon
Hobbies = final_Hobbies.drop('id')
X = np.array(Hobbies.select('features','Hobbies_quantity').collect())

from sklearn.model_selection import TimeSeriesSplit
from sklearn import datasets, ensemble
splits = TimeSeriesSplit(n_splits=3)
numTrees = [5,15,30,50]
maxDepth = [5,10,15]
RF_feature_list = []
for train_index, test_index in splits.split(X):
  train = X[train_index]
  test = X[test_index]
  train_df_pyspark = final_Hobbies.where(col("id").between(0, len(train))).drop('id')
  test_df_pyspark = final_Hobbies.where(col("id").between(len(train)+1, len(train)+len(test))).drop('id')
  for i in numTrees:
    for j in maxDepth:
      RF = RandomForestRegressor(labelCol="Hobbies_quantity",numTrees = i, maxDepth = j)
      pipeline = Pipeline(stages = [feature_Indexer, RF])
      model = pipeline.fit(train_df_pyspark)
      predictions = model.transform(test_df_pyspark)
      e = RegressionEvaluator(metricName="r2", labelCol=RF.getLabelCol(), predictionCol=RF.getPredictionCol())
      r2= e.evaluate(predictions)
      A = np.array(predictions.select('Hobbies_quantity').collect())
      F = np.array(predictions.select('prediction').collect())
      #Print RMSE for the current regressor along with the regression model  
      s = smape(A,F)
      f = [i,j,r2,s]
      RF_feature_list.append(f)
RF_feature_list

Out[21]: [[5, 5, 0.31856145939461955, 15.865795424113859],
 [5, 10, 0.25422241936447487, 16.57220254025985],
 [5, 15, 0.23579077980963392, 16.850257934413605],
 [15, 5, 0.33139840692746314, 15.80174677721801],
 [15, 10, 0.3165977993543756, 16.07321836866739],
 [15, 15, 0.320463856808327, 15.993084951140284],
 [30, 5, 0.2540094959894994, 16.85199921313285],
 [30, 10, 0.25635874834259964, 16.80349119655851],
 [30, 15, 0.25363552064600847, 16.86717292653452],
 [50, 5, 0.28365160028422975, 16.52105654155416],
 [50, 10, 0.27841376264902284, 16.616095404196415],
 [50, 15, 0.28305868859905914, 16.504969704884196],
 [5, 5, 0.4091157502851729, 9.349944174133663],
 [5, 10, 0.3541064810848604, 10.166102959641362],
 [5, 15, 0.31530693594422177, 10.55238106418354],
 [15, 5, 0.4415748506065137, 9.020152173349448],
 [15, 10, 0.45451435070331725, 8.81564230205926],
 [15, 15, 0.4597184989081583, 8.760753777155793],
 [30, 5, 0.40015364536669173, 9.290474179841185],
 [30, 10, 0.4042827757919266, 9.212231745711916],
 [30, 15, 0.4034410862681903, 9.216849558104313],
 [50, 5, 0.466897594626972, 8.728911962976177],
 [50, 10, 0.4858669935944129, 8.366030125865795],
 [50, 15, 0.487885791781018, 8.348711349086072],
 [5, 5, -0.1251778107586179, 13.26134613305761],
 [5, 10, -0.13585558271012643, 13.387393835071586],
 [5, 15, -0.11205229041001163, 13.080404187775066],
 [15, 5, 0.13831572162069228, 10.773454741556797],
 [15, 10, 0.17156298893745148, 10.499929796838364],
 [15, 15, 0.15375011386200388, 10.597790729845658],
 [30, 5, 0.15580193244005114, 10.873885399102472],
 [30, 10, 0.15586249120793438, 10.750572676151355],
 [30, 15, 0.15442655859614918, 10.734749426956986],
 [50, 5, 0.17910875443811702, 10.919803070934657],
 [50, 10, 0.20600738098452576, 10.658730346421587],
 [50, 15, 0.1782004537143579, 10.874912476716]]

###### FOODS : 1 DAY

In [35]:
# Specify Input Columns that will become features in our final models
featuresCols = F_day.columns
featuresCols.remove('lag_Foods')

#Build Features using the feature assembler
feature_assembler = VectorAssembler(inputCols = featuresCols, outputCol = "features")
output = feature_assembler.transform(F_day) # Transform dataframe into features
final_Foods = output.select("features", "lag_Foods") # Generate final output two cols: one with features and the other with the target i.e Foods

#Rename Hobbies Column to Hobbies_quantity to use with ML models in PySpark
final_Foods = final_Foods.select(col("features"),col("lag_Foods").alias("Foods_quantity"))

#Build Feature Indexer:
feature_Indexer = VectorIndexer(inputCol = 'features', outputCol = 'indexedFeatures',maxCategories = 12).fit(final_Foods)

final_Foods = final_Foods.select("*").withColumn("id", monotonically_increasing_id())

In [36]:
#GBT for Foods 1 day horizon
Foods = final_Foods.drop('id')
X = np.array(Foods.select('features','Foods_quantity').collect())

from sklearn.model_selection import TimeSeriesSplit
from sklearn import datasets, ensemble
splits = TimeSeriesSplit(n_splits=3)
maxDepth = (2,5)
maxIter = (10,20)
GBT_feature_list = []
for train_index, test_index in splits.split(X):                
  train = X[train_index]
  test = X[test_index]
  train_df_pyspark = final_Foods.where(col("id").between(0, len(train))).drop('id')
  test_df_pyspark = final_Foods.where(col("id").between(len(train)+1, len(train)+len(test))).drop('id')
  for i in maxDepth:
    for j in maxIter:
      GBT = GBTRegressor(labelCol="Foods_quantity",maxDepth = i, maxIter = j)
      pipeline = Pipeline(stages = [feature_Indexer, GBT])
      model = pipeline.fit(train_df_pyspark)
      predictions = model.transform(test_df_pyspark)
      e = RegressionEvaluator(metricName="r2", labelCol=GBT.getLabelCol(), predictionCol=GBT.getPredictionCol())
      r2= e.evaluate(predictions)
      A = np.array(predictions.select('Foods_quantity').collect())
      F = np.array(predictions.select('prediction').collect())
      #Print R2 and SMAPE for the current regressor along with the regression model  
      s = smape(A,F)
      f = [i,j,r2,s]
      GBT_feature_list.append(f)
GBT_feature_list 

Out[23]: [[2, 10, 0.45681518805234056, 10.434284817610253],
 [2, 20, 0.5314612799287665, 9.570639450916772],
 [5, 10, 0.45252217668526673, 9.98638951035706],
 [5, 20, 0.4499739361178048, 10.056393102477166],
 [2, 10, 0.4911546216035566, 9.923279445455847],
 [2, 20, 0.5258260263115194, 9.453251925428736],
 [5, 10, 0.4856884576338578, 9.377850464595797],
 [5, 20, 0.5085558077698067, 9.068971662118539],
 [2, 10, 0.5429393715567481, 9.373486235045258],
 [2, 20, 0.592805365496343, 8.789360734017308],
 [5, 10, 0.5984932375200205, 8.501767414702417],
 [5, 20, 0.4707975921175662, 10.293254537812357]]

In [37]:
#RF for Foods 1 day horizon
Foods = final_Foods.drop('id')
X = np.array(Foods.select('features','Foods_quantity').collect())

from sklearn.model_selection import TimeSeriesSplit
from sklearn import datasets, ensemble
splits = TimeSeriesSplit(n_splits=3)
numTrees = [5,15,30,50]
maxDepth = [5,10,15]
RF_feature_list = []
for train_index, test_index in splits.split(X):
  train = X[train_index]
  test = X[test_index]
  train_df_pyspark = final_Foods.where(col("id").between(0, len(train))).drop('id')
  test_df_pyspark = final_Foods.where(col("id").between(len(train)+1, len(train)+len(test))).drop('id')
  for i in numTrees:
    for j in maxDepth:
      RF = RandomForestRegressor(labelCol="Foods_quantity",numTrees = i, maxDepth = j)
      pipeline = Pipeline(stages = [feature_Indexer, RF])
      model = pipeline.fit(train_df_pyspark)
      predictions = model.transform(test_df_pyspark)
      e = RegressionEvaluator(metricName="r2", labelCol=RF.getLabelCol(), predictionCol=RF.getPredictionCol())
      r2= e.evaluate(predictions)
      A = np.array(predictions.select('Foods_quantity').collect())
      F = np.array(predictions.select('prediction').collect())
      #Print R2 and SMAPE for the current regressor along with the regression model  
      s = smape(A,F)
      f = [i,j,r2,s]
      RF_feature_list.append(f)
RF_feature_list

Out[24]: [[5, 5, 0.5021942534843311, 9.915769601537844],
 [5, 10, 0.47386260116996837, 10.37377473300008],
 [5, 15, 0.472981047140804, 10.32812418141099],
 [15, 5, 0.5333241410515916, 9.497490198638332],
 [15, 10, 0.5590868564668466, 9.241178788988378],
 [15, 15, 0.5593531392923479, 9.252563756740038],
 [30, 5, 0.5452904394985674, 9.323526816493429],
 [30, 10, 0.5634301372050026, 9.26610824724838],
 [30, 15, 0.5628820794801785, 9.27479673312933],
 [50, 5, 0.5290480754116486, 9.477345157888648],
 [50, 10, 0.580579359451034, 8.770511049815658],
 [50, 15, 0.5805413555981338, 8.766499586184668],
 [5, 5, 0.5655526816840359, 8.871205408428892],
 [5, 10, 0.5772672350275978, 8.437754163158345],
 [5, 15, 0.5609290200636194, 8.551519906719875],
 [15, 5, 0.582165580862346, 8.696751066479631],
 [15, 10, 0.6236242812165478, 7.939473147970429],
 [15, 15, 0.6183513291581353, 8.014792499238995],
 [30, 5, 0.5884351707993408, 8.590140678055285],
 [30, 10, 0.6285063923256282, 7.940184476511972],
 [30, 15, 0.6258071331011359, 7.954687081574346],
 [50, 5, 0.6002686713679883, 8.422216819502827],
 [50, 10, 0.6410633369390542, 7.747725395336474],
 [50, 15, 0.6364255079990971, 7.798713089306006],
 [5, 5, 0.6089995348662088, 8.34535645506378],
 [5, 10, 0.6619497198517652, 7.706579696194804],
 [5, 15, 0.65427018297073, 7.873158678015978],
 [15, 5, 0.6170087706642493, 8.094263775386974],
 [15, 10, 0.6619364835606831, 7.495955377424763],
 [15, 15, 0.6544937533718116, 7.47025243493615],
 [30, 5, 0.6229082317424757, 8.002293554810187],
 [30, 10, 0.6553647827922195, 7.372342414762536],
 [30, 15, 0.6697583612845219, 7.216653802081899],
 [50, 5, 0.6011245448230029, 8.250228954649959],
 [50, 10, 0.6652880052857473, 7.387806117723431],
 [50, 15, 0.671977310442402, 7.250594206691029]]

###### FOODS : 7 DAY

In [39]:
# Specify Input Columns that will become features in our final models
featuresCols = F_week.columns
featuresCols.remove('lag_Foods')

#Build Features using the feature assembler
feature_assembler = VectorAssembler(inputCols = featuresCols, outputCol = "features")
output = feature_assembler.transform(F_week) # Transform dataframe into features
final_Foods = output.select("features", "lag_Foods") # Generate final output two cols: one with features and the other with the target i.e Foods

#Rename Hobbies Column to Hobbies_quantity to use with ML models in PySpark
final_Foods = final_Foods.select(col("features"),col("lag_Foods").alias("Foods_quantity"))

#Build Feature Indexer:
feature_Indexer = VectorIndexer(inputCol = 'features', outputCol = 'indexedFeatures',maxCategories = 12).fit(final_Foods)

final_Foods = final_Foods.select("*").withColumn("id", monotonically_increasing_id())

In [40]:
#GBT for Foods 7 days horizon
Foods = final_Foods.drop('id')
X = np.array(Foods.select('features','Foods_quantity').collect())

from sklearn.model_selection import TimeSeriesSplit
from sklearn import datasets, ensemble
splits = TimeSeriesSplit(n_splits=3)
maxDepth = (2,5)
maxIter = (10,20)
GBT_feature_list = []
for train_index, test_index in splits.split(X):                
  train = X[train_index]
  test = X[test_index]
  train_df_pyspark = final_Foods.where(col("id").between(0, len(train))).drop('id')
  test_df_pyspark = final_Foods.where(col("id").between(len(train)+1, len(train)+len(test))).drop('id')
  for i in maxDepth:
    for j in maxIter:
      GBT = GBTRegressor(labelCol="Foods_quantity",maxDepth = i, maxIter = j)
      pipeline = Pipeline(stages = [feature_Indexer, GBT])
      model = pipeline.fit(train_df_pyspark)
      predictions = model.transform(test_df_pyspark)
      e = RegressionEvaluator(metricName="r2", labelCol=GBT.getLabelCol(), predictionCol=GBT.getPredictionCol())
      r2= e.evaluate(predictions)
      A = np.array(predictions.select('Foods_quantity').collect())
      F = np.array(predictions.select('prediction').collect())
      #Print R2 and SMAPE for the current regressor along with the regression model  
      s = smape(A,F)
      f = [i,j,r2,s]
      GBT_feature_list.append(f)
GBT_feature_list 

Out[26]: [[2, 10, 0.4372822355025562, 10.406954927642712],
 [2, 20, 0.4573051078575693, 10.201613255543965],
 [5, 10, 0.41867774519512746, 10.752208150817975],
 [5, 20, 0.3841279139841782, 11.02506614511922],
 [2, 10, 0.45111966514839985, 9.89260423301973],
 [2, 20, 0.48193755538431815, 9.665694840369953],
 [5, 10, 0.4369237618384628, 10.003664225226501],
 [5, 20, 0.44816380809718415, 9.915483449651324],
 [2, 10, 0.44329774742763517, 10.138837102980164],
 [2, 20, 0.3796609326319559, 11.17139851117568],
 [5, 10, 0.3106162895024527, 11.710454440068379],
 [5, 20, 0.2702875713726617, 12.19979791556385]]

In [41]:
#RF for Foods 7 days horizon
Foods = final_Foods.drop('id')
X = np.array(Foods.select('features','Foods_quantity').collect())

from sklearn.model_selection import TimeSeriesSplit
from sklearn import datasets, ensemble
splits = TimeSeriesSplit(n_splits=3)
numTrees = [5,15,30,50]
maxDepth = [5,10,15]
RF_feature_list = []
for train_index, test_index in splits.split(X):
  train = X[train_index]
  test = X[test_index]
  train_df_pyspark = final_Foods.where(col("id").between(0, len(train))).drop('id')
  test_df_pyspark = final_Foods.where(col("id").between(len(train)+1, len(train)+len(test))).drop('id')
  for i in numTrees:
    for j in maxDepth:
      RF = RandomForestRegressor(labelCol="Foods_quantity",numTrees = i, maxDepth = j)
      pipeline = Pipeline(stages = [feature_Indexer, RF])
      model = pipeline.fit(train_df_pyspark)
      predictions = model.transform(test_df_pyspark)
      e = RegressionEvaluator(metricName="r2", labelCol=RF.getLabelCol(), predictionCol=RF.getPredictionCol())
      r2= e.evaluate(predictions)
      A = np.array(predictions.select('Foods_quantity').collect())
      F = np.array(predictions.select('prediction').collect())
      #Print R2 and SMAPE for the current regressor along with the regression model  
      s = smape(A,F)
      f = [i,j,r2,s]
      RF_feature_list.append(f)
RF_feature_list

Out[27]: [[5, 5, 0.4476041628425679, 10.691788975038035],
 [5, 10, 0.4397223231945475, 10.77706729592779],
 [5, 15, 0.4190234531605078, 10.985003530581285],
 [15, 5, 0.5223898845668415, 9.65606831123809],
 [15, 10, 0.5286770750303161, 9.533768321174536],
 [15, 15, 0.5266522226073027, 9.544563171314211],
 [30, 5, 0.5345506028187426, 9.720804282278845],
 [30, 10, 0.5377343241744845, 9.505671889967328],
 [30, 15, 0.5376364928774002, 9.50148702385306],
 [50, 5, 0.4863288448369779, 10.20424694247414],
 [50, 10, 0.5086508109976596, 9.835588626442775],
 [50, 15, 0.507697811156718, 9.835186350435455],
 [5, 5, 0.5149267980515719, 9.422312819819266],
 [5, 10, 0.2779290052565485, 12.230531909252297],
 [5, 15, 0.27537694462165774, 12.315156450668509],
 [15, 5, 0.5552106160065894, 9.108909599882006],
 [15, 10, 0.5497314427000786, 9.00685293591514],
 [15, 15, 0.5375108657650931, 9.086785075623027],
 [30, 5, 0.5662860772448215, 8.949139340118482],
 [30, 10, 0.5646798834579672, 8.726995649962717],
 [30, 15, 0.5567486446602162, 8.783437987576287],
 [50, 5, 0.5816547849927989, 8.761627339355174],
 [50, 10, 0.5971666015596427, 8.346128470644514],
 [50, 15, 0.5894273947349291, 8.35786773336],
 [5, 5, 0.5039536060401504, 9.522591886238777],
 [5, 10, 0.47415477964492136, 9.725689419969868],
 [5, 15, 0.46592826436810053, 9.785908437008207],
 [15, 5, 0.5038781848835066, 9.319856428727533],
 [15, 10, 0.5098358715895686, 9.287832386833346],
 [15, 15, 0.5055821502599844, 9.297101973960288],
 [30, 5, 0.4895447731253486, 9.56534243924945],
 [30, 10, 0.4973893279130991, 9.35891306687756],
 [30, 15, 0.4962254408005685, 9.427547169829923],
 [50, 5, 0.49319495985285455, 9.541616424759312],
 [50, 10, 0.49420265860423196, 9.365748845507769],
 [50, 15, 0.4905237316268716, 9.388939429123955]]

###### FOODS : 28 DAY

In [43]:
# Specify Input Columns that will become features in our final models
featuresCols = F_month.columns
featuresCols.remove('lag_Foods')

#Build Features using the feature assembler
feature_assembler = VectorAssembler(inputCols = featuresCols, outputCol = "features")
output = feature_assembler.transform(F_month) # Transform dataframe into features
final_Foods = output.select("features", "lag_Foods") # Generate final output two cols: one with features and the other with the target i.e Foods

#Rename Hobbies Column to Hobbies_quantity to use with ML models in PySpark
final_Foods = final_Foods.select(col("features"),col("lag_Foods").alias("Foods_quantity"))

#Build Feature Indexer:
feature_Indexer = VectorIndexer(inputCol = 'features', outputCol = 'indexedFeatures',maxCategories = 12).fit(final_Foods)

final_Foods = final_Foods.select("*").withColumn("id", monotonically_increasing_id())

In [44]:
#GBT for Foods 28 days horizon
Foods = final_Foods.drop('id')
X = np.array(Foods.select('features','Foods_quantity').collect())

from sklearn.model_selection import TimeSeriesSplit
from sklearn import datasets, ensemble
splits = TimeSeriesSplit(n_splits=3)
maxDepth = (2,5)
maxIter = (10,20)
GBT_feature_list = []
for train_index, test_index in splits.split(X):                
  train = X[train_index]
  test = X[test_index]
  train_df_pyspark = final_Foods.where(col("id").between(0, len(train))).drop('id')
  test_df_pyspark = final_Foods.where(col("id").between(len(train)+1, len(train)+len(test))).drop('id')
  for i in maxDepth:
    for j in maxIter:
      GBT = GBTRegressor(labelCol="Foods_quantity",maxDepth = i, maxIter = j)
      pipeline = Pipeline(stages = [feature_Indexer, GBT])
      model = pipeline.fit(train_df_pyspark)
      predictions = model.transform(test_df_pyspark)
      e = RegressionEvaluator(metricName="r2", labelCol=GBT.getLabelCol(), predictionCol=GBT.getPredictionCol())
      r2= e.evaluate(predictions)
      A = np.array(predictions.select('Foods_quantity').collect())
      F = np.array(predictions.select('prediction').collect())
      #Print R2 and SMAPE for the current regressor along with the regression model  
      s = smape(A,F)
      f = [i,j,r2,s]
      GBT_feature_list.append(f)
GBT_feature_list 

Out[29]: [[2, 10, 0.4220254470904279, 10.898053856724115],
 [2, 20, 0.3940652055735624, 11.04710915626987],
 [5, 10, 0.4003474551529925, 10.933069550682406],
 [5, 20, 0.4119963871650033, 10.841146036795339],
 [2, 10, 0.43168471122573926, 10.2487815247482],
 [2, 20, 0.46512008412715156, 9.876185025418916],
 [5, 10, 0.5158529002567017, 9.506922288710783],
 [5, 20, 0.5077516557881083, 9.441113128683558],
 [2, 10, 0.5169386962991356, 9.542730201975559],
 [2, 20, 0.5645034690179741, 8.933859564209907],
 [5, 10, 0.5184596868647622, 9.54599608345911],
 [5, 20, 0.5072387779018279, 9.680369760191859]]

In [45]:
#RF for Foods 28 days horizon
Foods = final_Foods.drop('id')
X = np.array(Foods.select('features','Foods_quantity').collect())

from sklearn.model_selection import TimeSeriesSplit
from sklearn import datasets, ensemble
splits = TimeSeriesSplit(n_splits=3)
numTrees = [5,15,30,50]
maxDepth = [5,10,15]
RF_feature_list = []
for train_index, test_index in splits.split(X):
  train = X[train_index]
  test = X[test_index]
  train_df_pyspark = final_Foods.where(col("id").between(0, len(train))).drop('id')
  test_df_pyspark = final_Foods.where(col("id").between(len(train)+1, len(train)+len(test))).drop('id')
  for i in numTrees:
    for j in maxDepth:
      RF = RandomForestRegressor(labelCol="Foods_quantity",numTrees = i, maxDepth = j)
      pipeline = Pipeline(stages = [feature_Indexer, RF])
      model = pipeline.fit(train_df_pyspark)
      predictions = model.transform(test_df_pyspark)
      e = RegressionEvaluator(metricName="r2", labelCol=RF.getLabelCol(), predictionCol=RF.getPredictionCol())
      r2= e.evaluate(predictions)
      A = np.array(predictions.select('Foods_quantity').collect())
      F = np.array(predictions.select('prediction').collect())
      #Print R2 and SMAPE for the current regressor along with the regression model  
      s = smape(A,F)
      f = [i,j,r2,s]
      RF_feature_list.append(f)
RF_feature_list

Out[30]: [[5, 5, 0.492812225490608, 10.347809064015951],
 [5, 10, 0.47033925922848685, 10.476741935159485],
 [5, 15, 0.4569652998874353, 10.64052767141032],
 [15, 5, 0.5440533234430749, 9.662442396093198],
 [15, 10, 0.5149438223879251, 10.169368874469384],
 [15, 15, 0.5151073217874416, 10.144538471704582],
 [30, 5, 0.548403778297792, 9.666661087430086],
 [30, 10, 0.5145694519394541, 10.198583544364373],
 [30, 15, 0.5129855564396655, 10.199381335715474],
 [50, 5, 0.5452974097229877, 9.683477116743195],
 [50, 10, 0.5079889561491295, 10.249666994045803],
 [50, 15, 0.5072144503624894, 10.253551847184864],
 [5, 5, 0.579487459355803, 8.745448463919324],
 [5, 10, 0.5854283322064406, 8.716295335951015],
 [5, 15, 0.5796040618664036, 8.856973936856788],
 [15, 5, 0.5797466260643278, 8.591302109421882],
 [15, 10, 0.5935160775884527, 8.237507003969588],
 [15, 15, 0.5936805288616949, 8.274802879241811],
 [30, 5, 0.5839935899563217, 8.520164584863322],
 [30, 10, 0.6080629301343004, 8.111432654769523],
 [30, 15, 0.6059795742735894, 8.068660085144607],
 [50, 5, 0.6012807776492537, 8.411321990276786],
 [50, 10, 0.6315843064242848, 7.942865946064033],
 [50, 15, 0.6310582818113248, 7.922715360779131],
 [5, 5, 0.5924465006499142, 8.62270562109344],
 [5, 10, 0.5978237744402021, 8.422442954743637],
 [5, 15, 0.5744932085558484, 8.714603979855314],
 [15, 5, 0.5993632658506874, 8.244191244426434],
 [15, 10, 0.6310605238398297, 7.861299836662527],
 [15, 15, 0.6268558831332729, 7.879512024613025],
 [30, 5, 0.6022372754686574, 8.371986042038236],
 [30, 10, 0.6307582979448123, 7.970656217093648],
 [30, 15, 0.630500284173678, 7.975562759030853],
 [50, 5, 0.6018383433144123, 8.343980571244561],
 [50, 10, 0.6362129422479295, 7.9559915627615485],
 [50, 15, 0.6404713615033528, 7.883356416109056]]

###### HOUSEHOLD : 1 DAY

In [47]:
# Specify Input Columns that will become features in our final models
featuresCols = HH_day.columns
featuresCols.remove('lag_HH')

#Build Features using the feature assembler
feature_assembler = VectorAssembler(inputCols = featuresCols, outputCol = "features")
output = feature_assembler.transform(HH_day) # Transform dataframe into features
final_HH = output.select("features", "lag_HH") # Generate final output two cols: one with features and the other with the target i.e HH

#Rename Hobbies Column to Hobbies_quantity to use with ML models in PySpark
final_HH = final_HH.select(col("features"),col("lag_HH").alias("HH_quantity"))

#Build Feature Indexer:
feature_Indexer = VectorIndexer(inputCol = 'features', outputCol = 'indexedFeatures',maxCategories = 12).fit(final_HH)

final_HH = final_HH.select("*").withColumn("id", monotonically_increasing_id())

In [48]:
#GBT for HouseHold 1 day horizon
HH = final_HH.drop('id')
X = np.array(HH.select('features','HH_quantity').collect())

from sklearn.model_selection import TimeSeriesSplit
from sklearn import datasets, ensemble
splits = TimeSeriesSplit(n_splits=3)
maxDepth = (2,5)
maxIter = (10,20)
GBT_feature_list = []
for train_index, test_index in splits.split(X):                
  train = X[train_index]
  test = X[test_index]
  train_df_pyspark = final_HH.where(col("id").between(0, len(train))).drop('id')
  test_df_pyspark = final_HH.where(col("id").between(len(train)+1, len(train)+len(test))).drop('id')
  for i in maxDepth:
    for j in maxIter:
      GBT = GBTRegressor(labelCol="HH_quantity",maxDepth = i, maxIter = j)
      pipeline = Pipeline(stages = [feature_Indexer, GBT])
      model = pipeline.fit(train_df_pyspark)
      predictions = model.transform(test_df_pyspark)
      e = RegressionEvaluator(metricName="r2", labelCol=GBT.getLabelCol(), predictionCol=GBT.getPredictionCol())
      r2= e.evaluate(predictions)
      A = np.array(predictions.select('HH_quantity').collect())
      F = np.array(predictions.select('prediction').collect())
      #Print R2 and SMAPE for the current regressor along with the regression model  
      s = smape(A,F)
      f = [i,j,r2,s]
      GBT_feature_list.append(f)
GBT_feature_list 

Out[32]: [[2, 10, 0.12587550142454473, 14.236753674782333],
 [2, 20, 0.16343606869730598, 14.020073878802178],
 [5, 10, -0.09914085042119547, 17.46797230428485],
 [5, 20, -0.050551752334226485, 17.197380683704523],
 [2, 10, 0.6011056077498015, 10.151282725898664],
 [2, 20, 0.6450036217645405, 9.567542880061037],
 [5, 10, 0.5187831913037437, 11.08040851341486],
 [5, 20, 0.49011461035506854, 11.349388356429447],
 [2, 10, 0.3774121015182892, 11.203950923477729],
 [2, 20, 0.4439913930084801, 10.457182827453002],
 [5, 10, 0.2523490796586001, 13.093460933955527],
 [5, 20, 0.3460104198516014, 12.087959656112734]]

In [49]:
#RF for HouseHold 1 day horizon
HH = final_HH.drop('id')
X = np.array(HH.select('features','HH_quantity').collect())

from sklearn.model_selection import TimeSeriesSplit
from sklearn import datasets, ensemble
splits = TimeSeriesSplit(n_splits=3)
numTrees = [5,15,30,50]
maxDepth = [5,10,15]
RF_feature_list = []
for train_index, test_index in splits.split(X):
  train = X[train_index]
  test = X[test_index]
  train_df_pyspark = final_HH.where(col("id").between(0, len(train))).drop('id')
  test_df_pyspark = final_HH.where(col("id").between(len(train)+1, len(train)+len(test))).drop('id')
  for i in numTrees:
    for j in maxDepth:
      RF = RandomForestRegressor(labelCol="HH_quantity",numTrees = i, maxDepth = j)
      pipeline = Pipeline(stages = [feature_Indexer, RF])
      model = pipeline.fit(train_df_pyspark)
      predictions = model.transform(test_df_pyspark)
      e = RegressionEvaluator(metricName="r2", labelCol=RF.getLabelCol(), predictionCol=RF.getPredictionCol())
      r2= e.evaluate(predictions)
      A = np.array(predictions.select('HH_quantity').collect())
      F = np.array(predictions.select('prediction').collect())
      #Print R2 and SMAPE for the current regressor along with the regression model  
      s = smape(A,F)
      f = [i,j,r2,s]
      RF_feature_list.append(f)
RF_feature_list

Out[33]: [[5, 5, -0.33204824911171116, 18.10877799033163],
 [5, 10, -0.1331859811299856, 16.699717427437708],
 [5, 15, -0.1330666747479914, 16.694737748854976],
 [15, 5, 0.03399553136323097, 14.319465594264194],
 [15, 10, 0.03372259871665528, 14.399039097175605],
 [15, 15, 0.030272683563406844, 14.424719284665269],
 [30, 5, 0.018720919345584286, 14.587331309537094],
 [30, 10, 0.02513609795277494, 14.591091799358834],
 [30, 15, 0.023491008782178135, 14.616402022039694],
 [50, 5, -0.018512230395830498, 14.677946545192436],
 [50, 10, 0.11108198701966487, 13.909347782820628],
 [50, 15, 0.10216876501114369, 13.984599221351548],
 [5, 5, 0.655253215149721, 9.047527887544824],
 [5, 10, 0.6473817054221158, 9.582462573789993],
 [5, 15, 0.6324564338788874, 9.758044339662254],
 [15, 5, 0.6988578896758532, 8.49415729264648],
 [15, 10, 0.6767817974505186, 8.948050511187208],
 [15, 15, 0.6679336893317753, 9.119887847976337],
 [30, 5, 0.6952398846620046, 8.51589373588008],
 [30, 10, 0.6964388271403397, 8.426002340951827],
 [30, 15, 0.6906054915597184, 8.538898825217734],
 [50, 5, 0.6865759623366006, 8.6864261742398],
 [50, 10, 0.6963187185826087, 8.577851466916316],
 [50, 15, 0.6932534932495356, 8.652817826391196],
 [5, 5, 0.1880814651905578, 13.600068154809527],
 [5, 10, 0.3451294417435534, 12.614438949672179],
 [5, 15, 0.3630112768302052, 12.248271830700395],
 [15, 5, 0.31186313795523646, 12.090621360195655],
 [15, 10, 0.45492610217367047, 10.641793086447192],
 [15, 15, 0.44508190374529666, 10.849517525015308],
 [30, 5, 0.3537558307927593, 11.550434642366058],
 [30, 10, 0.4876636842123303, 10.51125016181022],
 [30, 15, 0.488899857288578, 10.513865491328065],
 [50, 5, 0.3327519336880571, 11.758306035770138],
 [50, 10, 0.5176516828521421, 9.931784594255388],
 [50, 15, 0.5178675325360074, 9.937254455016193]]

###### HOUSEHOLD : 7 DAY

In [51]:
# Specify Input Columns that will become features in our final models
featuresCols = HH_week.columns
featuresCols.remove('lag_HH')

#Build Features using the feature assembler
feature_assembler = VectorAssembler(inputCols = featuresCols, outputCol = "features")
output = feature_assembler.transform(HH_week) # Transform dataframe into features
final_HH = output.select("features", "lag_HH") # Generate final output two cols: one with features and the other with the target i.e HH

#Rename Hobbies Column to Hobbies_quantity to use with ML models in PySpark
final_HH = final_HH.select(col("features"),col("lag_HH").alias("HH_quantity"))

#Build Feature Indexer:
feature_Indexer = VectorIndexer(inputCol = 'features', outputCol = 'indexedFeatures',maxCategories = 12).fit(final_HH)

final_HH = final_HH.select("*").withColumn("id", monotonically_increasing_id())

In [52]:
#GBT for HouseHold 7 days horizon
HH = final_HH.drop('id')
X = np.array(HH.select('features','HH_quantity').collect())

from sklearn.model_selection import TimeSeriesSplit
from sklearn import datasets, ensemble
splits = TimeSeriesSplit(n_splits=3)
maxDepth = (2,5)
maxIter = (10,20)
GBT_feature_list = []
for train_index, test_index in splits.split(X):                
  train = X[train_index]
  test = X[test_index]
  train_df_pyspark = final_HH.where(col("id").between(0, len(train))).drop('id')
  test_df_pyspark = final_HH.where(col("id").between(len(train)+1, len(train)+len(test))).drop('id')
  for i in maxDepth:
    for j in maxIter:
      GBT = GBTRegressor(labelCol="HH_quantity",maxDepth = i, maxIter = j)
      pipeline = Pipeline(stages = [feature_Indexer, GBT])
      model = pipeline.fit(train_df_pyspark)
      predictions = model.transform(test_df_pyspark)
      e = RegressionEvaluator(metricName="r2", labelCol=GBT.getLabelCol(), predictionCol=GBT.getPredictionCol())
      r2= e.evaluate(predictions)
      A = np.array(predictions.select('HH_quantity').collect())
      F = np.array(predictions.select('prediction').collect())
      #Print R2 and SMAPE for the current regressor along with the regression model  
      s = smape(A,F)
      f = [i,j,r2,s]
      GBT_feature_list.append(f)
GBT_feature_list 

Out[35]: [[2, 10, 0.1178242309269012, 15.314066450100729],
 [2, 20, 0.12623207295231453, 15.542367917765977],
 [5, 10, 0.1750613914461755, 15.234655094783237],
 [5, 20, 0.17077111940973266, 15.27237555636729],
 [2, 10, 0.6250665689296759, 10.201641257988658],
 [2, 20, 0.6594828117874911, 9.634306079735097],
 [5, 10, 0.5961691266998315, 9.884372040771604],
 [5, 20, 0.5963600161121267, 9.884585981618695],
 [2, 10, 0.36155877764229294, 11.37908094622951],
 [2, 20, 0.39674871952414725, 10.954641872225256],
 [5, 10, 0.38072868874084265, 11.71523953562693],
 [5, 20, 0.3630818490032638, 11.895498428870148]]

In [53]:
#RF for HouseHold 7 days horizon
HH = final_HH.drop('id')
X = np.array(HH.select('features','HH_quantity').collect())

from sklearn.model_selection import TimeSeriesSplit
from sklearn import datasets, ensemble
splits = TimeSeriesSplit(n_splits=3)
numTrees = [5,15,30,50]
maxDepth = [5,10,15]
RF_feature_list = []
for train_index, test_index in splits.split(X):
  train = X[train_index]
  test = X[test_index]
  train_df_pyspark = final_HH.where(col("id").between(0, len(train))).drop('id')
  test_df_pyspark = final_HH.where(col("id").between(len(train)+1, len(train)+len(test))).drop('id')
  for i in numTrees:
    for j in maxDepth:
      RF = RandomForestRegressor(labelCol="HH_quantity",numTrees = i, maxDepth = j)
      pipeline = Pipeline(stages = [feature_Indexer, RF])
      model = pipeline.fit(train_df_pyspark)
      predictions = model.transform(test_df_pyspark)
      e = RegressionEvaluator(metricName="r2", labelCol=RF.getLabelCol(), predictionCol=RF.getPredictionCol())
      r2= e.evaluate(predictions)
      A = np.array(predictions.select('HH_quantity').collect())
      F = np.array(predictions.select('prediction').collect())
      #Print R2 and SMAPE for the current regressor along with the regression model  
      s = smape(A,F)
      f = [i,j,r2,s]
      RF_feature_list.append(f)
RF_feature_list

Out[36]: [[5, 5, -0.14742191511558134, 15.727825474227492],
 [5, 10, 0.10427105122658575, 14.950088333772491],
 [5, 15, 0.10338737933787012, 14.94681217463403],
 [15, 5, 0.22776223631944337, 14.293169524171574],
 [15, 10, 0.21060255845731257, 14.556168751996774],
 [15, 15, 0.21109353718826784, 14.54883811054313],
 [30, 5, 0.13793932410041843, 13.976612552075544],
 [30, 10, 0.1521050467255768, 13.932034405648212],
 [30, 15, 0.15132700079517825, 13.944049453904508],
 [50, 5, 0.19886308119510465, 13.94537921366033],
 [50, 10, 0.2453443655814168, 13.87975442150689],
 [50, 15, 0.24662587394913627, 13.866505991206147],
 [5, 5, 0.6877578267972725, 8.77412087885763],
 [5, 10, 0.6562527785535031, 9.032885621764388],
 [5, 15, 0.6495975423489713, 9.13088910987803],
 [15, 5, 0.7004829756920701, 8.794611082653176],
 [15, 10, 0.7025430080644135, 8.91181720038103],
 [15, 15, 0.7036285791468551, 8.903250469189619],
 [30, 5, 0.7003273492838981, 8.75069290592057],
 [30, 10, 0.7049129637550808, 8.759967868409374],
 [30, 15, 0.7049416801195055, 8.809731605654306],
 [50, 5, 0.7060462211080398, 8.63663023061683],
 [50, 10, 0.7139143782099606, 8.574318601012342],
 [50, 15, 0.7121097754311261, 8.634488706792634],
 [5, 5, 0.2835347190322127, 11.878200200812502],
 [5, 10, 0.3274774598189105, 12.194273384643907],
 [5, 15, 0.32569023195648206, 12.133802002632784],
 [15, 5, 0.3614553986505675, 11.279515237415549],
 [15, 10, 0.4598481448408599, 10.456758327426769],
 [15, 15, 0.45132047907290307, 10.482473298976224],
 [30, 5, 0.4179788404668221, 10.560553280169211],
 [30, 10, 0.5386519636020308, 9.446622210181431],
 [30, 15, 0.5478252652376391, 9.425906030297702],
 [50, 5, 0.4151056279319727, 10.445581499544923],
 [50, 10, 0.5489916524989518, 9.289631149926699],
 [50, 15, 0.5399588619177131, 9.35152892802411]]

###### HOUSEHOLD : 28 DAY

In [55]:
# Specify Input Columns that will become features in our final models
featuresCols = HH_month.columns
featuresCols.remove('lag_HH')

#Build Features using the feature assembler
feature_assembler = VectorAssembler(inputCols = featuresCols, outputCol = "features")
output = feature_assembler.transform(HH_month) # Transform dataframe into features
final_HH = output.select("features", "lag_HH") # Generate final output two cols: one with features and the other with the target i.e HH

#Rename Hobbies Column to Hobbies_quantity to use with ML models in PySpark
final_HH = final_HH.select(col("features"),col("lag_HH").alias("HH_quantity"))

#Build Feature Indexer:
feature_Indexer = VectorIndexer(inputCol = 'features', outputCol = 'indexedFeatures',maxCategories = 12).fit(final_HH)

final_HH = final_HH.select("*").withColumn("id", monotonically_increasing_id())

In [56]:
#GBT for HouseHold 28 days horizon
HH = final_HH.drop('id')
X = np.array(HH.select('features','HH_quantity').collect())

from sklearn.model_selection import TimeSeriesSplit
from sklearn import datasets, ensemble
splits = TimeSeriesSplit(n_splits=3)
maxDepth = (2,5)
maxIter = (10,20)
GBT_feature_list = []
for train_index, test_index in splits.split(X):                
  train = X[train_index]
  test = X[test_index]
  train_df_pyspark = final_HH.where(col("id").between(0, len(train))).drop('id')
  test_df_pyspark = final_HH.where(col("id").between(len(train)+1, len(train)+len(test))).drop('id')
  for i in maxDepth:
    for j in maxIter:
      GBT = GBTRegressor(labelCol="HH_quantity",maxDepth = i, maxIter = j)
      pipeline = Pipeline(stages = [feature_Indexer, GBT])
      model = pipeline.fit(train_df_pyspark)
      predictions = model.transform(test_df_pyspark)
      e = RegressionEvaluator(metricName="r2", labelCol=GBT.getLabelCol(), predictionCol=GBT.getPredictionCol())
      r2= e.evaluate(predictions)
      A = np.array(predictions.select('HH_quantity').collect())
      F = np.array(predictions.select('prediction').collect())
      #Print R2 and SMAPE for the current regressor along with the regression model  
      s = smape(A,F)
      f = [i,j,r2,s]
      GBT_feature_list.append(f)
GBT_feature_list 

Out[38]: [[2, 10, 0.03356757735500493, 16.35194511551509],
 [2, 20, 0.18270310455636174, 15.200914337936114],
 [5, 10, -0.0766650233851407, 16.85943568279473],
 [5, 20, -0.28016054687952563, 18.628261235227228],
 [2, 10, 0.5391522175197907, 11.091539378430054],
 [2, 20, 0.6005075830386208, 10.425637298621492],
 [5, 10, 0.48097498192211685, 11.49576895515757],
 [5, 20, 0.49000764771775684, 11.504683421968217],
 [2, 10, 0.1976833699816788, 13.112005648634707],
 [2, 20, 0.297803573701666, 12.12502178521586],
 [5, 10, 0.34803144567072997, 11.569703989986046],
 [5, 20, 0.37428849403263864, 11.218983140504031]]

In [57]:
#RF for HouseHold 28 days horizon
HH = final_HH.drop('id')
X = np.array(HH.select('features','HH_quantity').collect())

from sklearn.model_selection import TimeSeriesSplit
from sklearn import datasets, ensemble
splits = TimeSeriesSplit(n_splits=3)
numTrees = [5,15,30,50]
maxDepth = [5,10,15]
RF_feature_list = []
for train_index, test_index in splits.split(X):
  train = X[train_index]
  test = X[test_index]
  train_df_pyspark = final_HH.where(col("id").between(0, len(train))).drop('id')
  test_df_pyspark = final_HH.where(col("id").between(len(train)+1, len(train)+len(test))).drop('id')
  for i in numTrees:
    for j in maxDepth:
      RF = RandomForestRegressor(labelCol="HH_quantity",numTrees = i, maxDepth = j)
      pipeline = Pipeline(stages = [feature_Indexer, RF])
      model = pipeline.fit(train_df_pyspark)
      predictions = model.transform(test_df_pyspark)
      e = RegressionEvaluator(metricName="r2", labelCol=RF.getLabelCol(), predictionCol=RF.getPredictionCol())
      r2= e.evaluate(predictions)
      A = np.array(predictions.select('HH_quantity').collect())
      F = np.array(predictions.select('prediction').collect())
      #Print R2 and SMAPE for the current regressor along with the regression model  
      s = smape(A,F)
      f = [i,j,r2,s]
      RF_feature_list.append(f)
RF_feature_list

Out[39]: [[5, 5, -0.1376300946189155, 16.161503627107443],
 [5, 10, -0.09950930267869063, 16.173823770778178],
 [5, 15, -0.09959169522822009, 16.175901602186084],
 [15, 5, -0.18074464139779378, 16.582196527952398],
 [15, 10, -0.38024060406631244, 17.894191976359764],
 [15, 15, -0.3810177657159426, 17.902515118341487],
 [30, 5, -0.33521124481474573, 17.395015024626947],
 [30, 10, -0.29849652982267094, 17.168940135578648],
 [30, 15, -0.2992005283642283, 17.183327386097993],
 [50, 5, -0.009178731079000801, 15.582826731670732],
 [50, 10, -0.03369370669744365, 15.817440084451059],
 [50, 15, -0.037592146008962324, 15.84699856088841],
 [5, 5, 0.6302564388807146, 9.784426490187377],
 [5, 10, 0.6192150270795724, 10.013699157618404],
 [5, 15, 0.6085939492784767, 10.038837485189609],
 [15, 5, 0.6152008557911934, 9.810336253724389],
 [15, 10, 0.6159402191433114, 9.910543350670828],
 [15, 15, 0.6109502332937407, 9.982233994472486],
 [30, 5, 0.6350471107433695, 9.573063630785175],
 [30, 10, 0.6519493823315139, 9.229743708300903],
 [30, 15, 0.6466791515046144, 9.27789617345481],
 [50, 5, 0.6339677277086335, 9.691427877533917],
 [50, 10, 0.6519947924880012, 9.459435392381454],
 [50, 15, 0.6504630696716168, 9.458541706018075],
 [5, 5, 0.1533219587877308, 13.313437491188878],
 [5, 10, 0.21168334226300056, 13.071266031547534],
 [5, 15, 0.1604170624501018, 13.444223151758763],
 [15, 5, 0.328917640127651, 11.431838967156656],
 [15, 10, 0.44024802486722747, 10.652314296038549],
 [15, 15, 0.473606715247577, 10.34084074547988],
 [30, 5, 0.33054495350289703, 11.675004876568272],
 [30, 10, 0.49304209103359486, 10.09867597487747],
 [30, 15, 0.5052376405459675, 9.905416713064957],
 [50, 5, 0.351488903537962, 11.150296626344499],
 [50, 10, 0.48722629563125863, 9.830288312390449],
 [50, 15, 0.49223645088698087, 9.719182491924592]]